<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li></ul></div>

# Imports

In [2]:
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from YelpAPI import get_my_api_key

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [331]:
api_key = get_my_api_key()

In [ ]:
def scrape_urls(url_list):
    restaurant_list = []
    for url in url_list:
        res = requests.get(url= url['url'])
        try:
            res.raise_for_status()
            for item in BeautifulSoup(markup=res.text, features= 'html.parser').body.select('strong [href]'):
                restaurant = item.get_text().replace('&','and')
                restaurant = restaurant.replace("’s",'s').replace("’N",'N').replace("’n",'n')
                restaurant_list.append({'restaurant': restaurant,
                                       'neighborhood': url['location']})
        except Exception:
            print(f'Your request raised an error: {Exception}')
    return restaurant_list

In [ ]:
def get_businesses(location, term):
    '''Returns businesses in the specified location that match the specified term'''
    endpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = dict(Authorization = 'Bearer %s' % api_key)
    business_data = []
    
    for city in location:
        for offset in range(0,1000,50):
            parameters = dict(term = term,
                     limit = 50,
                    offset = offset,
                     radius = 40000,
                     location = location)
            response = requests.get(url = endpoint, params = parameters, headers = headers)
            if response.status_code == 200:
                business_data += response.json()['businesses']
            elif response.status_code == 400:
                print('400 Error: Bad Request')
                break
    return business_data

In [ ]:
def search_businesses(list_of_businesses):
    '''Takes a dictionary of restaurants names and their locations and searches for them in Yelp'''
    endpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = dict(Authorization = 'Bearer %s' % api_key)
    business_data = []
    unmatched_businesses = []
    data_history = []
    
    for item in list_of_businesses:
        parameters = dict(term = item['restaurant'],
                         limit = 1,
                          radius = 40000,
                         location = item['neighborhood'])
        response = requests.get(url = endpoint, params = parameters, headers = headers)
        
        if response.status_code == 200:
            if response.json()['businesses']:
                business_data += response.json()['businesses']
                data_history.append('Your search {} returned {}'.format(item['restaurant'],response.json()['businesses'][0]['name'] ))
            else:
                print('{} in {} not found in Yelp'.format(item['restaurant'], item['neighborhood']))
                unmatched_businesses.append({'restaurant': item['restaurant'], 'neighborhood': item['neighborhood']})
                continue
        elif response.status_code == 400:
            print('400 Error: Bad Request')
            break
    return business_data, unmatched_businesses, data_history

In [ ]:
def search_businesses2(list_of_businesses):
    '''Takes a dictionary of restaurants names and their locations and searches for them in Yelp'''
    endpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = dict(Authorization = 'Bearer %s' % api_key)
    business_data = []
    unmatched_businesses2 = []
    for item in list_of_businesses:
        parameters = dict(term = item['restaurant'],
                         limit = 1,
                          radius = 40000,
                         location = 'Los Angeles')
        response = requests.get(url = endpoint, params = parameters, headers = headers)
        
        if response.status_code == 200:
            if response.json()['businesses']:
                business_data += response.json()['businesses']
            else:
                print('{} in {} not found in Yelp'.format(item['restaurant'], item['neighborhood']))
                unmatched_businesses2.append({'restaurant': item['restaurant'], 'neighborhood': item['neighborhood']})
                continue
        elif response.status_code == 400:
            print('400 Error: Bad Request')
            break
    return business_data, unmatched_businesses2

In [272]:
infatuation_urls = [{'location': 'NYC', 'url': 'https://www.theinfatuation.com/features/support-nyc-black-owned-restaurants'},
                   {'location': 'Philadelphia', 'url': 'https://www.theinfatuation.com/features/support-philadelphias-black-owned-restaurants'},
                   {'location': 'Boston', 'url': 'https://www.theinfatuation.com/features/boston-black-owned-restaurants'},
                   {'location': 'Chicago', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-chicago'},
                   {'location': 'Miami', 'url': 'https://www.theinfatuation.com/features/miami-black-owned-restaurants'},
                   {'location': 'Seattle', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-seattle'},
                   {'location': 'Los Angeles', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-la'},
                   {'location': 'San Francisco Bay Area', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-the-bay-area'},
                   {'location': 'Austin', 'url': 'https://www.theinfatuation.com/features/austin-black-owned-restaurants-food-trucks'}]

In [276]:
print(infatuation_urls[:-1])

[{'location': 'NYC', 'url': 'https://www.theinfatuation.com/features/support-nyc-black-owned-restaurants'}, {'location': 'Philadelphia', 'url': 'https://www.theinfatuation.com/features/support-philadelphias-black-owned-restaurants'}, {'location': 'Boston', 'url': 'https://www.theinfatuation.com/features/boston-black-owned-restaurants'}, {'location': 'Chicago', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-chicago'}, {'location': 'Miami', 'url': 'https://www.theinfatuation.com/features/miami-black-owned-restaurants'}, {'location': 'Seattle', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-seattle'}, {'location': 'Los Angeles', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-la'}, {'location': 'San Francisco Bay Area', 'url': 'https://www.theinfatuation.com/features/support-black-owned-restaurants-in-the-bay-area'}]


In [284]:
def scrape_urls(url_list):
    restaurant_list = []
    for url in url_list:
        res = requests.get(url= url['url'])
        try:
            res.raise_for_status()
            for item in BeautifulSoup(markup=res.text, features= 'html.parser').body.select('strong [href]'):
                restaurant = item.get_text().replace('&','and')
                restaurant = restaurant.replace("’s",'s').replace("’N",'N').replace("’n",'n')
                restaurant_list.append({'restaurant': restaurant,
                                       'neighborhood': url['location']})
        except Exception:
            print(f'Your request raised an error: {Exception}')
    return restaurant_list

In [285]:
infatuation_restaurants = scrape_urls(infatuation_urls[:-1])

In [286]:
infatuation_restaurants

[{'restaurant': 'Tings Jamaican Jerk Chicken', 'neighborhood': 'NYC'},
 {'restaurant': 'Mikey Likes It East Village', 'neighborhood': 'NYC'},
 {'restaurant': 'Brooklyn Chop House', 'neighborhood': 'NYC'},
 {'restaurant': 'Alibi Lounge', 'neighborhood': 'NYC'},
 {'restaurant': 'Benyam', 'neighborhood': 'NYC'},
 {'restaurant': 'BLVD Bistro', 'neighborhood': 'NYC'},
 {'restaurant': 'Charles’ Pan Fried Chicken', 'neighborhood': 'NYC'},
 {'restaurant': 'Chocolat', 'neighborhood': 'NYC'},
 {'restaurant': 'The Edge Harlem', 'neighborhood': 'NYC'},
 {'restaurant': 'Famous Fish Market', 'neighborhood': 'NYC'},
 {'restaurant': 'Field Trip', 'neighborhood': 'NYC'},
 {'restaurant': 'Harlem Hops', 'neighborhood': 'NYC'},
 {'restaurant': 'Home Sweet Harlem', 'neighborhood': 'NYC'},
 {'restaurant': 'Kingston', 'neighborhood': 'NYC'},
 {'restaurant': 'Lee Lees Rugelach', 'neighborhood': 'NYC'},
 {'restaurant': 'Lolos Seafood Shack', 'neighborhood': 'NYC'},
 {'restaurant': 'Melbas', 'neighborhood': 'NY

In [287]:
matched_businesses, unmatched_businesses, check = search_businesses(infatuation_restaurants)

DaleView Biscuits and Beer in NYC not found in Yelp
Mac’n By Mari in Philadelphia not found in Yelp
Daaiyahs Delicious in Philadelphia not found in Yelp
Sahabah Cafe in Philadelphia not found in Yelp
Honeysuckle Popup in Philadelphia not found in Yelp
Jr. Billys Smokehouse in Philadelphia not found in Yelp
Rons Carribbean Cafe in Philadelphia not found in Yelp
Zaneyahs Delights in Philadelphia not found in Yelp
Pretzel Workz in Philadelphia not found in Yelp
Chef Reekys in Philadelphia not found in Yelp
Keebooms Kitchen in Philadelphia not found in Yelp
Saddiqs Water Ice in Philadelphia not found in Yelp
Natifnatal in Boston not found in Yelp
Raphael Carribean in Boston not found in Yelp
Vaughn Fish and Chips in Boston not found in Yelp
RandS Jamaican in Boston not found in Yelp
PandR Jamaican in Boston not found in Yelp
Shenger Cafe and Ethiopian in Boston not found in Yelp
PandR Jamaican in Boston not found in Yelp
BandB Ice Cream And Candy in Chicago not found in Yelp
Phlavz Bar And

In [288]:
len(matched_businesses)

1144

In [303]:
len(unmatched_businesses)

55

In [289]:
check

['Your search Tings Jamaican Jerk Chicken returned Tings Jamaican Jerk Chicken',
 'Your search Mikey Likes It East Village returned Snowdays',
 'Your search Brooklyn Chop House returned Brooklyn Chop House',
 'Your search Alibi Lounge returned Alibi',
 'Your search Benyam returned Benyam Cuisine',
 'Your search BLVD Bistro returned BLVD Bistro',
 'Your search Charles’ Pan Fried Chicken returned Charles Pan Fried Chicken',
 'Your search Chocolat returned Chocolat Restaurant & Bar',
 'Your search The Edge Harlem returned The Edge Harlem',
 'Your search Famous Fish Market returned Famous Fish Market',
 'Your search Field Trip returned Fieldtrip',
 'Your search Harlem Hops returned Harlem Hops',
 'Your search Home Sweet Harlem returned Home Sweet Harlem',
 'Your search Kingston returned Kingston',
 "Your search Lee Lees Rugelach returned Lee Lee's Baked Goods",
 "Your search Lolos Seafood Shack returned Lolo's Seafood Shack",
 "Your search Melbas returned Melba's",
 'Your search 67 Orange 

In [337]:
# endpoint = 'https://api.yelp.com/v3/businesses/search'
# headers = dict(Authorization = 'Bearer %s' % api_key)
# manually_added = []

# parameters = dict(term = 'Tana Ethiopian',
#                  limit = 5,
#                  radius = 40000,
#                  location = 'Anaheim')
# response = requests.get(url = endpoint, params = parameters, headers = headers)
# response.json()['businesses']

[{'id': 'iEu07v6D8uXyMOr12V4pRA',
  'alias': 'tana-ethiopian-restaurant-anaheim',
  'name': 'Tana Ethiopian Restaurant',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/WpQLDRoTiif9rpjL687f4w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/tana-ethiopian-restaurant-anaheim?adjust_creative=TbeWL7dBz_EmPK89bHN42A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TbeWL7dBz_EmPK89bHN42A',
  'review_count': 348,
  'categories': [{'alias': 'ethiopian', 'title': 'Ethiopian'}],
  'rating': 4.0,
  'coordinates': {'latitude': 33.846273, 'longitude': -117.977206},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '2622 W La Palma Ave',
   'address2': '',
   'address3': '',
   'city': 'Anaheim',
   'zip_code': '92801',
   'country': 'US',
   'state': 'CA',
   'display_address': ['2622 W La Palma Ave', 'Anaheim, CA 92801']},
  'phone': '+17142291719',
  'display_phone': '(714) 229-1719',
  'distance': 7154.3103304895985},
 {'id'

In [338]:
# matched_businesses.append(response.json()['businesses'][0])

In [304]:
unmatched_businesses

[{'restaurant': 'DaleView Biscuits and Beer', 'neighborhood': 'NYC'},
 {'restaurant': 'Mac’n By Mari', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Daaiyahs Delicious', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Sahabah Cafe', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Honeysuckle Popup', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Jr. Billys Smokehouse', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Rons Carribbean Cafe', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Zaneyahs Delights', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Pretzel Workz', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Chef Reekys', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Keebooms Kitchen', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Saddiqs Water Ice', 'neighborhood': 'Philadelphia'},
 {'restaurant': 'Natifnatal', 'neighborhood': 'Boston'},
 {'restaurant': 'Raphael Carribean', 'neighborhood': 'Boston'},
 {'restaurant': 'Vaughn Fish and Chips', 'neighborh

In [353]:
returned_businesses[0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [354]:
column_headers = ['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance']

In [352]:
matched_businesses[-1].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance'])

In [383]:
# Turn the list of dictionaries into a pandas DataFrame
businesses = pd.DataFrame(matched_businesses, columns = column_headers)
businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0sNH2OU3RiV3uf52M20L1Q,tings-jamaican-jerk-chicken-new-york,Tings Jamaican Jerk Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/KH2D5D...,False,https://www.yelp.com/biz/tings-jamaican-jerk-c...,42,"[{'alias': 'caribbean', 'title': 'Caribbean'},...",3.5,"{'latitude': 40.7422926, 'longitude': -74.0061...","[pickup, delivery]",NaN,"{'address1': '75 9th Ave', 'address2': None, '...",,,4216.963539
1,7jXNphhpZZ2YE37E9jP60g,snowdays-new-york,Snowdays,https://s3-media2.fl.yelpcdn.com/bphoto/KclcWx...,False,https://www.yelp.com/biz/snowdays-new-york?adj...,1157,"[{'alias': 'shavedice', 'title': 'Shaved Ice'}...",4.0,"{'latitude': 40.7290310768554, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '241 E 10th St', 'address2': None...",+12129828881,(212) 982-8881,2743.751946
2,A0saPXtRGKl-SVJGQyQAOw,brooklyn-chop-house-new-york-3,Brooklyn Chop House,https://s3-media1.fl.yelpcdn.com/bphoto/v_C5TF...,False,https://www.yelp.com/biz/brooklyn-chop-house-n...,434,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.0,"{'latitude': 40.7115099, 'longitude': -74.00597}","[delivery, pickup]",$$$,"{'address1': '150 Nassau St', 'address2': None...",+12126191200,(212) 619-1200,1187.511006
3,jv_nLKw5_PFjFhFsoKwcYw,alibi-new-york-4,Alibi,https://s3-media1.fl.yelpcdn.com/bphoto/67RRd8...,False,https://www.yelp.com/biz/alibi-new-york-4?adju...,45,"[{'alias': 'lounges', 'title': 'Lounges'}]",2.5,"{'latitude': 40.817856, 'longitude': -73.942294}",[],$$,"{'address1': '2376 Adam Clayton Powell', 'addr...",+19174727789,(917) 472-7789,13244.626462
4,2tTUnW5filzR3YG-G7rZ0w,benyam-cuisine-new-york,Benyam Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/W5C0XT...,False,https://www.yelp.com/biz/benyam-cuisine-new-yo...,117,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",5.0,"{'latitude': 40.82522, 'longitude': -73.94058}",[delivery],$$,"{'address1': '2795 Frederick Douglas Blvd', 'a...",+12125107353,(212) 510-7353,14066.802202


In [384]:
businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1149 non-null   object 
 1   alias          1149 non-null   object 
 2   name           1149 non-null   object 
 3   image_url      1149 non-null   object 
 4   is_closed      1149 non-null   bool   
 5   url            1149 non-null   object 
 6   review_count   1149 non-null   int64  
 7   categories     1149 non-null   object 
 8   rating         1149 non-null   float64
 9   coordinates    1149 non-null   object 
 10  transactions   1149 non-null   object 
 11  price          920 non-null    object 
 12  location       1149 non-null   object 
 13  phone          1149 non-null   object 
 14  display_phone  1149 non-null   object 
 15  distance       1149 non-null   float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 135.9+ KB


In [385]:
businesses.dtypes

id                object
alias             object
name              object
image_url         object
is_closed           bool
url               object
review_count       int64
categories        object
rating           float64
coordinates       object
transactions      object
price             object
location          object
phone             object
display_phone     object
distance         float64
dtype: object

In [386]:
businesses['is_closed'] = businesses['is_closed'].astype('category')
businesses['price'] = businesses['price'].astype('category')

In [387]:
businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,0sNH2OU3RiV3uf52M20L1Q,tings-jamaican-jerk-chicken-new-york,Tings Jamaican Jerk Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/KH2D5D...,False,https://www.yelp.com/biz/tings-jamaican-jerk-c...,42,"[{'alias': 'caribbean', 'title': 'Caribbean'},...",3.5,"{'latitude': 40.7422926, 'longitude': -74.0061...","[pickup, delivery]",NaN,"{'address1': '75 9th Ave', 'address2': None, '...",,,4216.963539
1,7jXNphhpZZ2YE37E9jP60g,snowdays-new-york,Snowdays,https://s3-media2.fl.yelpcdn.com/bphoto/KclcWx...,False,https://www.yelp.com/biz/snowdays-new-york?adj...,1157,"[{'alias': 'shavedice', 'title': 'Shaved Ice'}...",4.0,"{'latitude': 40.7290310768554, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '241 E 10th St', 'address2': None...",+12129828881,(212) 982-8881,2743.751946
2,A0saPXtRGKl-SVJGQyQAOw,brooklyn-chop-house-new-york-3,Brooklyn Chop House,https://s3-media1.fl.yelpcdn.com/bphoto/v_C5TF...,False,https://www.yelp.com/biz/brooklyn-chop-house-n...,434,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.0,"{'latitude': 40.7115099, 'longitude': -74.00597}","[delivery, pickup]",$$$,"{'address1': '150 Nassau St', 'address2': None...",+12126191200,(212) 619-1200,1187.511006
3,jv_nLKw5_PFjFhFsoKwcYw,alibi-new-york-4,Alibi,https://s3-media1.fl.yelpcdn.com/bphoto/67RRd8...,False,https://www.yelp.com/biz/alibi-new-york-4?adju...,45,"[{'alias': 'lounges', 'title': 'Lounges'}]",2.5,"{'latitude': 40.817856, 'longitude': -73.942294}",[],$$,"{'address1': '2376 Adam Clayton Powell', 'addr...",+19174727789,(917) 472-7789,13244.626462
4,2tTUnW5filzR3YG-G7rZ0w,benyam-cuisine-new-york,Benyam Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/W5C0XT...,False,https://www.yelp.com/biz/benyam-cuisine-new-yo...,117,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",5.0,"{'latitude': 40.82522, 'longitude': -73.94058}",[delivery],$$,"{'address1': '2795 Frederick Douglas Blvd', 'a...",+12125107353,(212) 510-7353,14066.802202


In [388]:
businesses['category'] = businesses['categories'].apply(pd.Series)[0].apply(pd.Series)['title']
businesses['sub_category'] = businesses['categories'].apply(pd.Series)[1].apply(pd.Series)['title']

In [389]:
businesses['latitude'] =businesses['coordinates'].apply(pd.Series).iloc[:,0]
businesses['longitude'] =businesses['coordinates'].apply(pd.Series).iloc[:,1]

In [390]:
business_locations = businesses['location'].apply(pd.Series)

In [391]:
business_locations

,address1,address2,address3,city,zip_code,country,state,display_address
0,75 9th Ave,None,None,New York,10011,US,NY,"[75 9th Ave, New York, NY 10011]"
1,241 E 10th St,None,,New York,10003,US,NY,"[241 E 10th St, New York, NY 10003]"
2,150 Nassau St,None,,New York,10038,US,NY,"[150 Nassau St, New York, NY 10038]"
3,2376 Adam Clayton Powell,,None,New York,10030,US,NY,"[2376 Adam Clayton Powell, New York, NY 10030]"
4,2795 Frederick Douglas Blvd,None,,New York,10039,US,NY,"[2795 Frederick Douglas Blvd, New York, NY 10039]"
...,...,...,...,...,...,...,...,...
1144,199 Avenue A,None,,New York,10009,US,NY,"[199 Avenue A, New York, NY 10009]"
1145,633 Nostrand Ave,None,None,Brooklyn,11216,US,NY,"[633 Nostrand Ave, Brooklyn, NY 11216]"
1146,66-48 Myrtle Ave,,,Glendale,11385,US,NY,"[66-48 Myrtle Ave, Glendale, NY 11385]"
1147,7304 Ogontz Ave,,,Philadelphia,19138,US,PA,"[7304 Ogontz Ave, Philadelphia, PA 19138]"


In [392]:
businesses = businesses.merge(business_locations, left_index = True, right_index = True)

In [393]:
businesses['display_address'] = businesses['display_address'].apply(lambda x: ', '.join(map(str,x)))

In [394]:
businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,category,sub_category,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address
0,0sNH2OU3RiV3uf52M20L1Q,tings-jamaican-jerk-chicken-new-york,Tings Jamaican Jerk Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/KH2D5D...,False,https://www.yelp.com/biz/tings-jamaican-jerk-c...,42,"[{'alias': 'caribbean', 'title': 'Caribbean'},...",3.5,"{'latitude': 40.7422926, 'longitude': -74.0061...","[pickup, delivery]",NaN,"{'address1': '75 9th Ave', 'address2': None, '...",,,4216.963539,Caribbean,Juice Bars & Smoothies,40.742293,-74.006131,75 9th Ave,None,None,New York,10011,US,NY,"75 9th Ave, New York, NY 10011"
1,7jXNphhpZZ2YE37E9jP60g,snowdays-new-york,Snowdays,https://s3-media2.fl.yelpcdn.com/bphoto/KclcWx...,False,https://www.yelp.com/biz/snowdays-new-york?adj...,1157,"[{'alias': 'shavedice', 'title': 'Shaved Ice'}...",4.0,"{'latitude': 40.7290310768554, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '241 E 10th St', 'address2': None...",+12129828881,(212) 982-8881,2743.751946,Shaved Ice,Ice Cream & Frozen Yogurt,40.729031,-73.984728,241 E 10th St,None,,New York,10003,US,NY,"241 E 10th St, New York, NY 10003"
2,A0saPXtRGKl-SVJGQyQAOw,brooklyn-chop-house-new-york-3,Brooklyn Chop House,https://s3-media1.fl.yelpcdn.com/bphoto/v_C5TF...,False,https://www.yelp.com/biz/brooklyn-chop-house-n...,434,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.0,"{'latitude': 40.7115099, 'longitude': -74.00597}","[delivery, pickup]",$$$,"{'address1': '150 Nassau St', 'address2': None...",+12126191200,(212) 619-1200,1187.511006,Dim Sum,Steakhouses,40.711510,-74.005970,150 Nassau St,None,,New York,10038,US,NY,"150 Nassau St, New York, NY 10038"
3,jv_nLKw5_PFjFhFsoKwcYw,alibi-new-york-4,Alibi,https://s3-media1.fl.yelpcdn.com/bphoto/67RRd8...,False,https://www.yelp.com/biz/alibi-new-york-4?adju...,45,"[{'alias': 'lounges', 'title': 'Lounges'}]",2.5,"{'latitude': 40.817856, 'longitude': -73.942294}",[],$$,"{'address1': '2376 Adam Clayton Powell', 'addr...",+19174727789,(917) 472-7789,13244.626462,Lounges,NaN,40.817856,-73.942294,2376 Adam Clayton Powell,,None,New York,10030,US,NY,"2376 Adam Clayton Powell, New York, NY 10030"
4,2tTUnW5filzR3YG-G7rZ0w,benyam-cuisine-new-york,Benyam Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/W5C0XT...,False,https://www.yelp.com/biz/benyam-cuisine-new-yo...,117,"[{'alias': 'ethiopian', 'title': 'Ethiopian'}]",5.0,"{'latitude': 40.82522, 'longitude': -73.94058}",[delivery],$$,"{'address1': '2795 Frederick Douglas Blvd', 'a...",+12125107353,(212) 510-7353,14066.802202,Ethiopian,NaN,40.825220,-73.940580,2795 Frederick Douglas Blvd,None,,New York,10039,US,NY,"2795 Frederick Douglas Blvd, New York, NY 10039"


In [395]:
businesses.state.value_counts()

CA    507
NY    176
PA    134
WA    118
IL     96
MA     65
FL     45
NJ      8
Name: state, dtype: int64

In [396]:
businesses.describe()

,review_count,rating,distance,latitude,longitude
count,1149.000000,1149.000000,1149.000000,1149.000000,1149.000000
mean,328.965187,4.068755,11351.461423,38.699746,-100.606830
std,615.565717,0.607422,10337.934170,4.764549,22.031875
min,1.000000,1.000000,246.004467,25.700904,-122.740959
25%,41.000000,3.500000,4795.171156,34.101762,-122.184906
50%,127.000000,4.000000,8252.988454,39.948581,-118.242872
75%,341.000000,4.500000,15153.356640,40.823550,-75.157584
max,8070.000000,5.000000,89007.836525,47.979564,-70.885940


In [397]:
businesses.set_index('id', inplace=True)

In [420]:
businesses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1098 entries, 0sNH2OU3RiV3uf52M20L1Q to iEu07v6D8uXyMOr12V4pRA
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             1098 non-null   object  
 1   image_url        1098 non-null   object  
 2   is_closed        1098 non-null   category
 3   url              1098 non-null   object  
 4   review_count     1098 non-null   int64   
 5   rating           1098 non-null   float64 
 6   transactions     1098 non-null   object  
 7   price            884 non-null    category
 8   display_phone    1098 non-null   object  
 9   category         1098 non-null   object  
 10  sub_category     728 non-null    object  
 11  latitude         1098 non-null   float64 
 12  longitude        1098 non-null   float64 
 13  address1         1079 non-null   object  
 14  address2         783 non-null    object  
 15  address3         828 non-null    object  
 16  city    

In [412]:
category_counts = businesses.category.value_counts()
correct_categroies = category_counts[category_counts > 1].index

In [414]:
businesses = businesses.loc[businesses.category.isin(correct_categroies)]

In [415]:
businesses.category.value_counts()

Caribbean                    110
Ethiopian                     91
Soul Food                     84
Coffee & Tea                  69
Southern                      58
Barbeque                      56
Seafood                       52
Bakeries                      37
Desserts                      33
African                       33
Vegan                         25
American (New)                24
Cajun/Creole                  24
Breakfast & Brunch            23
American (Traditional)        23
Food Trucks                   22
Chicken Wings                 21
Caterers                      20
Juice Bars & Smoothies        20
Bars                          17
Ice Cream & Frozen Yogurt     16
Cafes                         14
Burgers                       14
Sandwiches                    12
Coffee Roasteries             10
Haitian                        9
Mexican                        8
Seafood Markets                8
Pizza                          8
Lounges                        7
Comfort Fo

In [ ]:
categories_to_drop = ['Vietnamese', 'Music Venues']

In [417]:
businesses.query('category == "Filipino"')

,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,category,sub_category,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
LbeL3KUv6JP298uGXxt_fg,the-lumpia-company-oakland,The Lumpia Company,https://s3-media4.fl.yelpcdn.com/bphoto/06izZs...,False,https://www.yelp.com/biz/the-lumpia-company-oa...,159,"[{'alias': 'filipino', 'title': 'Filipino'}, {...",4.0,"{'latitude': 37.813402, 'longitude': -122.2644...",[delivery],$$,"{'address1': '372 24th St', 'address2': None, ...",+15108355885,(510) 835-5885,5373.509762,Filipino,Pop-up Shops,37.813402,-122.264494,372 24th St,None,None,Oakland,94612,US,CA,"372 24th St, Oakland, CA 94612"
3XexEOwxeBI2pbVZ3Tl6fQ,tastebuds-san-bruno,Tastebuds,https://s3-media3.fl.yelpcdn.com/bphoto/3ZK37_...,False,https://www.yelp.com/biz/tastebuds-san-bruno?a...,356,"[{'alias': 'filipino', 'title': 'Filipino'}, {...",3.0,"{'latitude': 37.62935, 'longitude': -122.41589}","[delivery, pickup]",$$,"{'address1': '582 San Bruno Ave W', 'address2'...",+16505830899,(650) 583-0899,26307.607041,Filipino,Noodles,37.629350,-122.415890,582 San Bruno Ave W,,,San Bruno,94066,US,CA,"582 San Bruno Ave W, San Bruno, CA 94066"


In [418]:
businesses.drop(columns = ['coordinates', 'location', 'alias', 'categories', 'phone', 'distance'], inplace=True)

In [419]:
businesses.to_csv('cleaned_businesses2.csv')